# Deploy Model Online

In [1]:
import pickle
import boto3

In [2]:
# Upload the model to aws bucket

s3 = boto3.client("s3")
s3.upload_file("home_purchase.pkl", "cortex-ee16ff5ab7", "home_purchase.pkl")

## Write Predictor Class

In [37]:
%%writefile predictor.py

import os
import pickle

labels = ["False", "True"]


class PythonPredictor:
    def __init__(self, config):
        """
        if os.environ.get("AWS_ACCESS_KEY_ID"):
            s3 = boto3.client("s3")  # client will use your credentials if available
        else:
            s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # anonymous client

        s3.download_file(config["bucket"], config["key"], "/tmp/model.pkl")
        """
        self.model = pickle.load(open("home_purchase.pkl", "rb"))

    def predict(self, payload):
   
        label_id = self.model.predict([payload['data']])[0]
        return labels[label_id]

Overwriting predictor.py


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../../assets/dataset/home_db.csv', index_col=None)
df.shape
df.head()

y = df.loc[:,'home_purchase']
X = df.loc[:,:'ACNT_SEC_CLOSE_*']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [11]:
# test predictor
payload = {'data': list(X_test.iloc[0].values)}
rf = PythonPredictor(None).model
rf.predict([payload['data']])

array([1])

## Deployment Configuration

In [18]:
%%writefile cortex.yaml
- name: home-purchase-model
  kind: RealtimeAPI
  predictor:
    type: python
    path: predictor.py
    config:
      bucket: cortex-ee16ff5ab7
      key: dev/lgbm_model.pkl

Writing cortex.yaml


## Deployment Requirements

In [19]:
%%writefile requirements.txt
pandas
scikit-learn
lightgbm

Writing requirements.txt


## Deploy Model
deploy the model in terminal

In [12]:
# !cortex deploy
# !cortex get 
# !cortex get recommender

## Query Deployed Model

### 1. Request

In [45]:
import requests 
  
# api-endpoint 
URL = "http://localhost:8888"

# send some queries
headers = {"Content-Type":"application/json"}
for i in range(10):
    p = {"data": [int(val) for val in X_test.iloc[i].values]}
    r = requests.post(url = URL, data = json.dumps(p), headers=headers)  
    print(r.text)

True
True
True
False
False
True
True
True
False
False


### 2. curl

In [17]:
payload

{'data': [1968, 4, 2, 0, 0, 0, 1, 0, 1, 0]}

In [18]:
%%writefile sample.json
{"data": [1968, 4, 2, 0, 0, 0, 1, 0, 1, 0]}

Overwriting sample.json


In [30]:
!curl http://localhost:8888 -X POST -H "Content-Type: application/json" -d @sample.json

False

In [55]:
!curl https://t0erni03x7.execute-api.us-west-1.amazonaws.com/gamification-model -X POST -H "Content-Type: application/json" -d @sample.json

True